In [21]:
import pandas as pd
import plotly
import plotly.express as  px
from plotly import colors
import textwrap

## Millenial seeks home built for this millenium
_San Francisco's aging housing stock just another consequence of its stalling housing pipeline_

For the first half of 2024, my partner and I were committed to the finanically dubious goal of buying a home in San Francisco. We hired a real estate agent, attended open houses most weekends, and developed unhealthy relationships with the Zillow app. What started out as a hopeful next chapter in our lives was quickly reality checked. Almost every unit that met our requirements was a very old home with some very old problems -- major fixers, to say the least. After repeatedly being quoted multiple hundreds of dollars in needed repairs, we simply gave up.

Our housing search faltered somewhat predictably, but what did surprise me is how much it turned me off to the charming old Victorians and Edwardians that characterize many SF neighborhoods. Normally I love the vibe of an old house, but our housing search taught me that with _charm_ and _character_ comes sloping floors, crumbling foundations, questionable electrical installations, furances from the 1940s, and on, and on. I wondered, where are the homes that just work? Are there any that don't need to be retrofitted to a new century?

In San Francisco, that type of home is a rare breed. Only about 12% of the city's housing stock was built in this millenium<sup>[1](#Percentage-of-housing-stock-built-since-2000)</sup>; 50,000 units, give or take. To put that number in perspective, the city of San Francisco needs to build 82,000 new housing units between 2023 and 2031 in order to meet its state-mandated Regional Housing Needs Allocation commitment<sup>[2](https://sfplanning.org/sites/default/files/documents/citywide/housing-choice/RHNA-Primer.pdf)</sup>. In just eight years, the city needs to build 1.5x the housing that it built in the last twenty-four.

Much has been said in the public discourse about San Francisco's urgent need to build more housing<sup>[3](https://www.sfchronicle.com/sf/article/housing-permits-san-francisco-17652633.php), [4](https://www.nytimes.com/2023/10/25/us/san-francisco-housing.html), [5](https://www.sfexaminer.com/news/housing/experts-say-sf-zoning-a-total-mismatch-from-housing-needs/article_b9d713fa-8996-11ee-aa28-6f28f6cdd4ab.html)</sup>. Since I moved here in 2019, the insistent fact of our failure to build has seemed the norm, an unfortunate but consistent reality pinning us to the status quo. You can imagine my surprise, then, when I saw that data on building permits<sup>[6](#Permit-Filing-and-Completion-Data)</sup> illustrated a clear downward trend in the rate of housing development. It doesn't look like we're adamantly holding to a too-slow pace. It looks like we're stalling out.

In [22]:
import warnings
warnings.filterwarnings('ignore')

permits_df = pd.read_csv("data/deduplicated_residential_building_permits_issued_filed_since_2000.csv")
# Convert dates to pandas datetime objects.
permits_df.loc[:, "Filed Date"] = pd.to_datetime(permits_df["Filed Date"])
permits_df.loc[:, "Issued Date"] = pd.to_datetime(permits_df["Issued Date"])
permits_df.loc[:, "Current Status Date"] = pd.to_datetime(permits_df["Current Status Date"])

The number of issued permits for new housing has been trending down since its peak in 2016. Fewer permits were issued to build new housing in 2023 than almost any other time since the turn of the millenium, save only for the years immediately following the 2008 housing bubble collapse.

In [23]:
plotly.offline.init_notebook_mode(connected=True)

issued_permits = permits_df[~permits_df["Issued Date"].isna()]
permits_issued_by_year = issued_permits.groupby(
    lambda idx: issued_permits.loc[idx]["Issued Date"].year
)["Permit Number"].count().drop(2024)  # Drop 2024, since the year isn't finished yet
fig = px.line(
    permits_issued_by_year,
    labels={"value": "Number of Permits Issued", "index": "Year"},
    width=800,
    template="plotly_white"
)
fig.update_layout(showlegend=False)


Perhaps even more troubling, the number of permit applications filed for new housing is also trending downard since its recent peak in 2015.

In [24]:
plotly.offline.init_notebook_mode(connected=True)

permits_filed_by_year = permits_df.groupby(
    lambda idx: int(permits_df.loc[idx]["Filed Date"].year)
)["Permit Number"].count()
fig = px.line(
    # Filter to after 2000, and drop 2024 since the year isn't over yet.
    permits_filed_by_year[
        (permits_filed_by_year.index >= 2000) &
        (permits_filed_by_year.index < 2024)
    ],
    labels={"value": "Number of Permits Filed", "index": "Year"}, 
    width=800, 
    template="plotly_white"
)
fig.update_layout(showlegend=False)

The permitting process itself is commonly blamed for discouraging new development. The process is famously complicated, expensive, and long. In this too, we are getting worse. The percentage of permits issued to new housing applications within two, three, and four years have all been declining since 2012.

In [25]:
def _make_percentage_of_permits_issued_in_n_years(n):
    def _fn(s):
        # Unissued permits, where Issued Date is NaN , will have a NaT duration,
        # whose days property is NaN, which is never <= C for any constant value C.
        issuing_duration = s["Issued Date"] - s["Filed Date"]
        issuing_duration_less_than_n_years = issuing_duration.apply(lambda x: x.days <= n * 365)
        percentage_issued_in_less_than_n_years = 100 * issuing_duration_less_than_n_years.astype(int).mean()
        return percentage_issued_in_less_than_n_years
    return _fn


def _get_approval_rates_over_waiting_periods(df_):
    grouped_by_file_year = df_.groupby(
        lambda idx: df_.loc[idx]["Filed Date"].year
    )
    percentage_of_permits_approved_in_two_years = grouped_by_file_year.apply(
        _make_percentage_of_permits_issued_in_n_years(2)
    ).drop([2022, 2023])
    percentage_of_permits_approved_in_three_years = grouped_by_file_year.apply(
        _make_percentage_of_permits_issued_in_n_years(3)
    ).drop([2021, 2022, 2023])
    percentage_of_permits_approved_in_four_years = grouped_by_file_year.apply(
        _make_percentage_of_permits_issued_in_n_years(4)
    ).drop([2020, 2021, 2022, 2023])

    approval_rates = pd.concat([
        percentage_of_permits_approved_in_two_years,
        percentage_of_permits_approved_in_three_years,
        percentage_of_permits_approved_in_four_years,
    ], axis=1, keys=["% Issued After 2 Years", "% Issued After 3 Years", "% Issued After 4 Years"])
    return approval_rates    

permits_filed_in_last_20_years = permits_df[permits_df["Filed Date"].apply(lambda x: x.year >= 2004 and x.year <= 2023)]
overall_approval_rates = _get_approval_rates_over_waiting_periods(permits_filed_in_last_20_years)
fig = px.line(overall_approval_rates, labels={"value": "% of Applications Issued Permits", "index": "Application Filing Year"}, width=800, template="plotly_white")

plotly.offline.init_notebook_mode(connected=True)
display(fig)

The Deputy Director of Permit Services, Neville Pereira, attributed the long wait times to the growing backlog of permit applications caused by lay-offs after the 2008 recession and disruptions from by COVID-19<sup>[7](https://www.sfchronicle.com/sf/article/housing-permits-san-francisco-17652633.php)</sup>. We can see the growing backlog as evidence of a decade long problem, starting in the early 2010s and exacerbated by the pandemic in 2020. The departments responsible for processing these applications clearly did not have enough resources to meet the demand. With such an evident signal of the problem, I'm left to wonder why it went on unabated for so long.

In [26]:
def _count_of_pending_permits_as_of_date(df, date):
    pending_permits = df[
        (df["Filed Date"] <= date)
        & ((df["Issued Date"].isna()) | (df["Issued Date"] > date))
        & ~(
            (df["Current Status"].isin(["withdrawn", "disapproved"]) )& 
            (df["Current Status Date"] <= date)
        )
    ]
    return len(pending_permits)


plotly.offline.init_notebook_mode(connected=True)
quarter_index = pd.date_range("2004-01-01", "2024-01-01", freq="QS")
backlog = quarter_index.to_series().apply(lambda x: _count_of_pending_permits_as_of_date(permits_df, x))
fig = px.line(
    backlog, 
    labels={"value": "Number of In-Progress Permit Applications", "index": "Time"}, 
    title="Permit Backlog at the start of each quarter",
    width=800,
    template="plotly_white"
)
fig.update_layout(showlegend=False)

When we zoom in on the timelines<sup>[8](#Permit-Routing-Data)</sup> of various types of residential developements, it's easy to see how an influx of new applications could cause a backup. The permitting process visualization looks as intimidatingly bureacratic as it is often described, involving around 10 separate city offices, routing tasks between each other and the applicant over months and years. During this time, lots sit under-utilized -- homes for cars instead of people. Simplifying a process is often easier said than done, but I think the need is evident.

In [27]:
import warnings
warnings.filterwarnings('ignore')

permit_routing_df = pd.read_csv("data/building_permit_adenda_with_routing_since_2014.csv")
permit_routing_df.loc[1273333, "start_date"] = "2024/04/22"  # Typo (previously 0202/04/22). 

# Convert strings to dates.
permit_routing_df.loc[:, "arrive"] = pd.to_datetime(permit_routing_df["arrive"])
permit_routing_df.loc[:, "start_date"] = pd.to_datetime(permit_routing_df["start_date"])
permit_routing_df.loc[:, "finish_date"] = pd.to_datetime(permit_routing_df["finish_date"])
permit_routing_df.loc[:, "in_hold"] = pd.to_datetime(permit_routing_df["in_hold"])
permit_routing_df.loc[:, "out_hold"] = pd.to_datetime(permit_routing_df["out_hold"])

In [29]:
permits_filed_after_2014_df = permits_df[permits_df["Filed Date"].apply(lambda x: x.year) >= 2014]
permits_filed_after_2014_routing_df = permits_filed_after_2014_df.merge(permit_routing_df, left_on="Permit Number", right_on="application_number")

In [40]:
def _find_permits_with_proposed_units(df, min_units, max_units, min_filing_year, min_issuing_days, max_issuing_days):
    return df[
        (df["Proposed Units"] >= min_units) &
        (df["Proposed Units"] < max_units) &
        (df["Filed Date"].apply(lambda x: x.year) >= min_filing_year) &
        ~(df["Issued Date"].isna()) &
        ((df["Issued Date"] - df["Filed Date"]).apply(lambda x: x.days) >= min_issuing_days) &
        ((df["Issued Date"] - df["Filed Date"]).apply(lambda x: x.days) < max_issuing_days)
    ]

def _permit_gannt_chart(df):
    station_code_to_name = {
        "CPB": "Central Permit Bureau",
        "CP-ZOC": "City Planning - Zoning Plan Clerk",
        "CP-NP": "City Planning - Residential Extension Notice",
        "BLDG": "Building Plan Review Services",
        "SFFD": "Fire Department",
        "DPW-BSM": "Bureau of Street Use and Mapping",
        "SFPUC": "Public Utilities Commission",
        "HEALTH": "Bureau of Health",
        "HEALTH - MH": "Bureau of Health - Mental Health Rehabilitation",
        "HEALTH - HM": "",
        "DPW-BUF": "Department of Public Works - Bureau of Urban Forestry",
        "DFCU": "Development Fee Collection Unit",
        "PPC": "Permit Processing Center",
    }
    df_copy = df[["department", "station", "step", "arrive", "start_date", "in_hold", "out_hold", "finish_date", "hold_description"]]
    df_copy["full_station_name"] = df_copy["station"].apply(
        lambda s: station_code_to_name[s] if s in station_code_to_name else s
    )

    pending = df_copy[["department", "full_station_name", "step", "arrive", "start_date", "in_hold"]].rename(columns=dict(arrive="start"))
    pending["end"] = pending["start_date"].where(
        (pending["in_hold"].isna()) | (pending["start_date"] <= pending["in_hold"]),
        pending["in_hold"]
    )
    pending.loc[:, "Phase"] = "Pending"

    pre_hold = df_copy[~df_copy["in_hold"].isna()][["department", "full_station_name", "step", "start_date", "in_hold"]].rename(columns=dict(start_date="start", in_hold="end"))
    pre_hold.loc[:, "Phase"] = "Started"

    # Consider end of "hold" to be either the "out_hold" date or "finish date", whichever is set.
    hold = df_copy[["department", "full_station_name", "step", "in_hold", "out_hold", "finish_date", "hold_description"]]
    hold["out_hold"] = hold["out_hold"].where(~hold["out_hold"].isna(), hold["finish_date"])
    hold = hold.rename(columns=dict(in_hold="start", out_hold="end"))
    hold.loc[:, "Phase"] = "In Hold"

    post_hold = df_copy[~df_copy["out_hold"].isna()][["department", "full_station_name", "step", "out_hold", "finish_date"]].rename(columns=dict(out_hold="start", finish_date="end"))
    post_hold.loc[:, "Phase"] = "Started"
    
    no_hold = df_copy[df_copy["in_hold"].isna()][["department", "full_station_name", "step", "start_date", "finish_date", "hold_description"]].rename(
        columns=dict(start_date="start", finish_date="end")
    )

    # Make sure the duration of this event is at least 1 day by adding a day if "start_date" and "finish_date" are the same.
    # This ensures the period is dispalyed on the chart, and can be hovered over to read the description.
    no_hold["end"] = no_hold["end"].where(no_hold["end"] > no_hold["start"], no_hold["end"] + pd.Timedelta(days=1))
    no_hold.loc[:, "Phase"] = "Started"

    staged_df = pd.concat(
        [pending, pre_hold, hold, post_hold, no_hold], axis=0
    ).sort_values(by=["step"])
    staged_df["hold_description"] = staged_df["hold_description"].apply(lambda s: "<br>".join(textwrap.wrap(s)) if isinstance(s, str) else "N/A")

    fig = px.timeline(
        staged_df,
        x_start="start",
        x_end="end",
        y="full_station_name",
        color="Phase",
        color_discrete_map={
            "Pending": colors.qualitative.Alphabet[24],  # yellow
            "Started": colors.qualitative.Alphabet[9],  # green
            "In Hold": colors.qualitative.Alphabet[17],  # red
        },
        hover_data="hold_description",
        title=f"{df["Street Number"].iloc[0]} {df["Street Name"].iloc[0]} {df["Street Suffix"].iloc[0]} ({int(df["Proposed Units"].iloc[0])} Units)",
        labels={"full_station_name": "Station"},
        category_orders={"Phase": ["Pending", "Started", "In Hold"]},
        template="plotly_white",
        width=1000
    )
    fig.update_yaxes(autorange="reversed")
    fig.show()

In [37]:
plotly.offline.init_notebook_mode(connected=True)

#_find_permits_with_proposed_units(permits_df, 2, 5, 2019, 365 * 2, 365 * 3)
_permit_gannt_chart(permits_filed_after_2014_routing_df[permits_filed_after_2014_routing_df["Permit Number"] == "202204071831"])

![428 15th Av](data/42815thave.png)

In [38]:
plotly.offline.init_notebook_mode(connected=True)

#_find_permits_with_proposed_units(permits_df, 10, 50, 2019, 365 * 2, 365 * 3)
_permit_gannt_chart(permits_filed_after_2014_routing_df[permits_filed_after_2014_routing_df["Permit Number"] == "201904299198"])

![1220 Valencia St](data/1120valenciast.png)

In [41]:
plotly.offline.init_notebook_mode(connected=True)

#_find_permits_with_proposed_units(permits_df, 100, float("inf"), 2019, 365 * 2, 365 * 3)
_permit_gannt_chart(permits_filed_after_2014_routing_df[permits_filed_after_2014_routing_df["Permit Number"] == "201912099034"])

![1567 California St](data/1567californiast.png)

Permitting data only shows a snapshot of the development pipeline. It doesn't speak to how restrictive zoning laws, expensive building costs, or high interest rates prevent projects from ever reaching the permiting stage. It doesn't tell us why projects fail to be built once permits are in order. What it does show is how insignificant action can lead to significant change. For more than a decade, we neglected to adequately respond to the clear signs of our city's growth, by either increasing staffing of the departments that handle these processes, or by simplifying the processes altogether. We are building fewer homes than we have in years in large part because we didn't choose to build more.

Recently, the California State government has started to intervene. SB-423<sup>[9](https://sfplanning.org/resource/sb423-application)</sup>, which went into effect in San Francisco in June, introduces a streamlined process for Planning Department approval which bypasses neighborhood notification and discretionary review, as well as mandates timelines of the Planning Department's review. AB-1114<sup>[10](https://sfplanning.org/updated-planning-approval)</sup> similarly regulates the review timelines for permits that are issued separately from Planning Department approval, including building permits issued by the Department of Building Inpsection. If these works as intended, we'll be estimating permitting times in months rather than years. 

Only time will tell if these interventions successfully lead to more new homes. I plan to wait it out in my comfortable rental apartment, hopes of homeownership put on ice. The next time I pass a human who, for lack of a better option, has made a parking lot their home, I'll remind myself that not everyone is so lucky.

***

#### Data Information

##### Percentage of housing stock built since 2000

The following counts the number of newly constructed residential units completed between January 1, 2000, and August 31, 2024. The data, with filters applied, was downloaded from [DataSF](https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9/explore/query/SELECT%0A%20%20%60permit_number%60%2C%0A%20%20%60permit_type%60%2C%0A%20%20%60permit_type_definition%60%2C%0A%20%20%60permit_creation_date%60%2C%0A%20%20%60block%60%2C%0A%20%20%60lot%60%2C%0A%20%20%60street_number%60%2C%0A%20%20%60street_number_suffix%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60street_suffix%60%2C%0A%20%20%60unit%60%2C%0A%20%20%60unit_suffix%60%2C%0A%20%20%60description%60%2C%0A%20%20%60status%60%2C%0A%20%20%60status_date%60%2C%0A%20%20%60filed_date%60%2C%0A%20%20%60issued_date%60%2C%0A%20%20%60completed_date%60%2C%0A%20%20%60first_construction_document_date%60%2C%0A%20%20%60structural_notification%60%2C%0A%20%20%60number_of_existing_stories%60%2C%0A%20%20%60number_of_proposed_stories%60%2C%0A%20%20%60voluntary_soft_story_retrofit%60%2C%0A%20%20%60fire_only_permit%60%2C%0A%20%20%60estimated_cost%60%2C%0A%20%20%60revised_cost%60%2C%0A%20%20%60existing_use%60%2C%0A%20%20%60existing_units%60%2C%0A%20%20%60proposed_use%60%2C%0A%20%20%60proposed_units%60%2C%0A%20%20%60plansets%60%2C%0A%20%20%60tidf_compliance%60%2C%0A%20%20%60existing_occupancy%60%2C%0A%20%20%60proposed_occupancy%60%2C%0A%20%20%60existing_construction_type%60%2C%0A%20%20%60existing_construction_type_description%60%2C%0A%20%20%60proposed_construction_type%60%2C%0A%20%20%60proposed_construction_type_description%60%2C%0A%20%20%60site_permit%60%2C%0A%20%20%60last_permit_activity_date%60%2C%0A%20%20%60application_submission_method%60%2C%0A%20%20%60adu%60%2C%0A%20%20%60primary_address_flag%60%2C%0A%20%20%60supervisor_district%60%2C%0A%20%20%60neighborhoods_analysis_boundaries%60%2C%0A%20%20%60zipcode%60%2C%0A%20%20%60location%60%2C%0A%20%20%60point_source%60%2C%0A%20%20%60record_id%60%2C%0A%20%20%60data_as_of%60%2C%0A%20%20%60data_loaded_at%60%0AWHERE%0A%20%20caseless_one_of%28%0A%20%20%20%20%60permit_type_definition%60%2C%0A%20%20%20%20%22new%20construction%22%2C%0A%20%20%20%20%22new%20construction%20wood%20frame%22%0A%20%20%29%0A%20%20AND%20%28caseless_one_of%28%0A%20%20%20%20%20%20%20%20%20%20%20%60proposed_use%60%2C%0A%20%20%20%20%20%20%20%20%20%20%20%221%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%222%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%22apartments%22%0A%20%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20%20AND%20%28%28%60proposed_units%60%20%3E%200%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%28caseless_one_of%28%60status%60%2C%20%22complete%22%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%28%60status_date%60%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20BETWEEN%20%222000-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%222024-08-31T23%3A59%3A59%22%20%3A%3A%20floating_timestamp%29%29%29%29/page/filter).

The number of total residential units in the housing stock (415,803) was pulled from the [2023 SF Housing Inventory Report](https://sfplanning.org/sites/default/files/resources/2024-04/2023_Housing_Inventory.pdf), page 15.

In [19]:
completed_residential_permits_df = pd.read_csv("data/deduplicated_residential_building_permits_completed_since_2000.csv")
num_completed_residential_units_since_2000 = completed_residential_permits_df["Proposed Units"].sum()
num_total_residential_units = 415_803
print("Number of completed units:", num_completed_residential_units_since_2000)
print("Percentage of housing stock:", f"{(100 * float(num_completed_residential_units_since_2000 / num_total_residential_units)):.02f}%")

Number of completed units: 50651
Percentage of housing stock: 12.18%


##### Permit Filing and Completion Data

The data used to create the graphs of permit issuings, permit application filings, wait times, and backlogs between January 1, 2000, and August 31, 2024 was downloaded, with filters applied, from [DataSF](https://data.sfgov.org/Housing-and-Buildings/Building-Permits-Deduplicated-on-Primary-Address/f2jc-ivnc/explore/query/SELECT%0A%20%20%60permit_number%60%2C%0A%20%20%60permit_type%60%2C%0A%20%20%60permit_type_definition%60%2C%0A%20%20%60permit_creation_date%60%2C%0A%20%20%60block%60%2C%0A%20%20%60lot%60%2C%0A%20%20%60street_number%60%2C%0A%20%20%60street_number_suffix%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60street_suffix%60%2C%0A%20%20%60unit%60%2C%0A%20%20%60unit_suffix%60%2C%0A%20%20%60description%60%2C%0A%20%20%60status%60%2C%0A%20%20%60status_date%60%2C%0A%20%20%60filed_date%60%2C%0A%20%20%60issued_date%60%2C%0A%20%20%60completed_date%60%2C%0A%20%20%60first_construction_document_date%60%2C%0A%20%20%60structural_notification%60%2C%0A%20%20%60number_of_existing_stories%60%2C%0A%20%20%60number_of_proposed_stories%60%2C%0A%20%20%60voluntary_soft_story_retrofit%60%2C%0A%20%20%60fire_only_permit%60%2C%0A%20%20%60estimated_cost%60%2C%0A%20%20%60revised_cost%60%2C%0A%20%20%60existing_use%60%2C%0A%20%20%60existing_units%60%2C%0A%20%20%60proposed_use%60%2C%0A%20%20%60proposed_units%60%2C%0A%20%20%60plansets%60%2C%0A%20%20%60tidf_compliance%60%2C%0A%20%20%60existing_occupancy%60%2C%0A%20%20%60proposed_occupancy%60%2C%0A%20%20%60existing_construction_type%60%2C%0A%20%20%60existing_construction_type_description%60%2C%0A%20%20%60proposed_construction_type%60%2C%0A%20%20%60proposed_construction_type_description%60%2C%0A%20%20%60site_permit%60%2C%0A%20%20%60last_permit_activity_date%60%2C%0A%20%20%60application_submission_method%60%2C%0A%20%20%60adu%60%2C%0A%20%20%60primary_address_flag%60%2C%0A%20%20%60supervisor_district%60%2C%0A%20%20%60neighborhoods_analysis_boundaries%60%2C%0A%20%20%60zipcode%60%2C%0A%20%20%60location%60%2C%0A%20%20%60point_source%60%2C%0A%20%20%60record_id%60%2C%0A%20%20%60data_as_of%60%2C%0A%20%20%60data_loaded_at%60%0AWHERE%0A%20%20%28caseless_one_of%28%0A%20%20%20%20%20%20%20%60permit_type_definition%60%2C%0A%20%20%20%20%20%20%20%22new%20construction%22%2C%0A%20%20%20%20%20%20%20%22new%20construction%20wood%20frame%22%0A%20%20%20%20%20%29%0A%20%20%20%20%20AND%20%28caseless_one_of%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%60proposed_use%60%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%221%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%222%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22apartments%22%0A%20%20%20%20%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20%20%20%20%20AND%20%28%60issued_date%60%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20BETWEEN%20%222000-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%222024-08-31T23%3A59%3A59%22%20%3A%3A%20floating_timestamp%29%29%29%0A%20%20%20%20OR%20%28caseless_one_of%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%60permit_type_definition%60%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%22new%20construction%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%22new%20construction%20wood%20frame%22%0A%20%20%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20%20%20AND%20%28caseless_one_of%28%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%60proposed_use%60%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%221%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%222%20family%20dwelling%22%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22apartments%22%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%29%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%28%60filed_date%60%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20BETWEEN%20%222000-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20AND%20%222024-08-31T23%3A59%3A59%22%20%3A%3A%20floating_timestamp%29%29%29/page/filter).

##### Permit Routing Data

The data used to create the timeline visualizations was downloaded, with filters applied, from [DataSF](https://data.sfgov.org/Housing-and-Buildings/Building-Permit-Addenda-with-Routing/87xy-gk8d/explore/query/SELECT%0A%20%20%60primary_key%60%2C%0A%20%20%60application_number%60%2C%0A%20%20%60addenda_number%60%2C%0A%20%20%60step%60%2C%0A%20%20%60station%60%2C%0A%20%20%60arrive%60%2C%0A%20%20%60assign_date%60%2C%0A%20%20%60start_date%60%2C%0A%20%20%60in_hold%60%2C%0A%20%20%60out_hold%60%2C%0A%20%20%60finish_date%60%2C%0A%20%20%60plan_checked_by%60%2C%0A%20%20%60hold_description%60%2C%0A%20%20%60title%60%2C%0A%20%20%60addenda_status%60%2C%0A%20%20%60department%60%2C%0A%20%20%60review_results%60%2C%0A%20%20%60data_as_of%60%2C%0A%20%20%60data_loaded_at%60%0AWHERE%0A%20%20%28%60arrive%60%0A%20%20%20%20%20BETWEEN%20%222014-01-01T00%3A00%3A00%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20%20AND%20%222024-09-16T13%3A55%3A11%22%20%3A%3A%20floating_timestamp%29%0A%20%20AND%20%28%60addenda_number%60%20%3D%200%29/page/filter).